# Project: Data Modeling with Apache Cassandra

## Part I: ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [73]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [74]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [75]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
#print(len(full_data_rows_list))
# see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [76]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Complete the Apache Cassandra coding portion of project. 

#### The CSV file titled <font color=red>event_datafile_new.csv</font> locates within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [77]:
# This should make a connection to a Cassandra instance local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [78]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songdb
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [79]:
# Set KEYSPACE to the keyspace specified above
try: 
    session.set_keyspace('songdb')
except Exception as e:
    print(e)

### Data Modeling with Apache Cassandra: the database tables will be modeled on the queries we want to run.

### QUERY 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### CREATE table song_info_by_session with primary key: (session_id, item_in_session) that uniquely identify songs' information

In [80]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### INSERT values into song_info_by_session table from event_datafile_new.csv file

In [81]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into song_info_by_session table

In [82]:
query = "SELECT session_id, item_in_session, artist, song_title, song_length FROM song_info_by_session WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Make use of the PrettyTables python library to present the results in a tabular format

In [83]:
result = PrettyTable()
result.field_names = ["session_id", "item_in_session", "artist", "song_title", "song_length"]
for row in rows:
    result.add_row([row.session_id, row.item_in_session, row.artist, row.song_title, row.song_length])
print(result)

+------------+-----------------+-----------+---------------------------------+--------------------+
| session_id | item_in_session |   artist  |            song_title           |    song_length     |
+------------+-----------------+-----------+---------------------------------+--------------------+
|    338     |        4        | Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+------------+-----------------+-----------+---------------------------------+--------------------+


### QUERY 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### CREATE table song_info_by_user with partition key (user_id, session_id) that uniquely identify songs' information and clustering key (item_in_session) by which song are sorted

In [84]:
query = """CREATE TABLE IF NOT EXISTS song_info_by_user 
           (user_id int, session_id int, artist text, song_title text, item_in_session int, first_name text, last_name text,
           PRIMARY KEY ((user_id, session_id), item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### INSERT values into song_info_by_user table from event_datafile_new.csv file

In [85]:
query = "INSERT INTO song_info_by_user (user_id, session_id, artist, song_title, item_in_session, first_name, last_name)"
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        try:
            session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into song_info_by_user table

In [86]:
query = "SELECT * FROM song_info_by_user WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Make use of the PrettyTables python library to present the results in a tabular format

In [87]:
result = PrettyTable()
result.field_names = ['artist', 'song_title', 'first_name', 'last_name']
for row in rows:
    result.add_row([row.artist, row.song_title, row.first_name, row.last_name])
print(result)

+-------------------+------------------------------------------------------+------------+-----------+
|       artist      |                      song_title                      | first_name | last_name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### QUERY 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### CREATE table user_name_by_song with primary key (song_title, user_id) that uniquely identify user names

In [88]:
query = """CREATE TABLE IF NOT EXISTS user_name_by_song
           (song_title text, user_id int, first_name text, last_name text,
           PRIMARY KEY (song_title, user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                

#### INSERT values into user_name_by_song table from event_datafile_new.csv file

In [89]:
query = "INSERT INTO user_name_by_song (song_title, user_id, first_name, last_name)"
query = query + "VALUES (%s, %s, %s, %s)"

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into user_name_by_song table

In [90]:
query = "SELECT song_title, first_name, last_name FROM user_name_by_song WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


#### Make use of the PrettyTables python library to present the results in a tabular format

In [91]:
result = PrettyTable()
result.field_names = ['song_title', 'first_name', 'last_name']
for row in rows:
    result.add_row([row.song_title, row.first_name, row.last_name])
print(result)

+------------+------------+-----------+
| song_title | first_name | last_name |
+------------+------------+-----------+
+------------+------------+-----------+


### Drop the tables before closing out the sessions

In [72]:
try:
    session.execute("DROP TABLE song_info_by_session")
except Exception as e:
    print(e)
try:
    session.execute("DROP TABLE song_info_by_user")
except Exception as e:
    print(e)
try:
    session.execute("DROP TABLE user_name_by_song")
except Exception as e:
    print(e)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


### Close the session and cluster connection¶

In [71]:
session.shutdown()
cluster.shutdown()